In [6]:
a=[0,1,2,3,4,5]
a[:3]

[0, 1, 2]

In [42]:
import random
# Node
class Node:
    def __init__(self, isleaf=False):
        self.isleaf=isleaf
        self.keys=[]
        self.child=[]
    def __repr__(self):
        return f'{self.keys}'

class BTree:
    def __init__(self, m):
        self.m=m #차수 -> m개의 child를 가질 수 있음
        self.root=Node(True)
        # self.root=None

    def printTree(self, node ,level=0):
        print(f'level {level} , length : {len(node.keys)} ', end=" : ")
        for i in node.keys: 
            print(i[0],end=" ")
        level+=1
        if len(node.child)>0:
            for i in node.child: 
                self.printTree(i, level)
        
    def splitNode(self,p_pos,p_node,node):
        pass

    # k : Key 
    # def insert_node(self,p_pos,p_node,node,key):
    def insert_node(self,node,key):
        """ 데이터 추가하는것, Root부터 시작해서 내려가도록 Trigger가 발동되며, 
        현재의 노드(node)가 leaf가 아닐경우, 재귀적으로 child를 내려감. 
        리프에서의 삽입이 이뤄지고나면, 그대로 자기자신만 반환하고, 
        Leaf에서의 삽입이 OverFlow가 일어날경우 중간key와 자기자신을 반환
        삽입을 하고난 이후, 반환되는 key가 있다면, 해당 child에서 OverFlow가 난것이므로, 관련연산 진행
        """
        print(key,node)
        pos=0
        while pos < len(node.keys)-2:
            print(pos)
            if key[0]==node.keys[pos]:
                #Duplicate
                print(f"Duplicates key {key}")
                return node, None 
            elif key[0]<node.keys[pos]: break
            pos+=1
        # Position 찾기 완료
        # print(pos)
        if node.isleaf: # Leaf라면?? -> Leaf에서의 삽입 로직
            i=len(node.keys)-1
            node.keys.append([1e9,None])
            while i>=0 and key[0]<node.keys[i][0]:
                node.keys[i+1]=node.keys[i]
                i-=1
            node.keys[i+1]=key
            if len(node.keys) >= self.m : # 오버플로 발생?
                return node, node.keys[(self.m//2)]
            else: return node, None
        else: #Leaf가 아니라면?
            node.child[pos], child_mid_value = self.insert_node(node.child[pos],key)
            if child_mid_value: # Child에서 OverFlow발생 -> 자기에 추가시켜줘야해
                # Parent로 올라가
                temp=node.keys[pos:]
                node.keys[pos+1:] = temp
                # node.keys[pos] = node.child.keys[child_mid_value]
                node.keys[pos] = child_mid_value
                node.child[pos+2:] = node.child[pos+1:]
                
                # child split
                c_node=node.child[pos]
                temp = Node(c_node.isleaf) # 오른쪽으로 추가될 노드
                temp.keys = c_node.keys[self.m//2+1:] ## m//2는 부모로 올라갔으므로
                c_node.keys=c_node.keys[:self.m//2]
                
                ## Child 의 child까지 정리
                node.child[pos+1] = temp
                if not c_node.isleaf:
                    temp.child = c_node.child[self.m//2+1:]
                    c_node.child = c_node.child[:self.m//2+1]
            
            if len(node.keys) >= self.m : # 오버플로 발생?
                return node, node.keys[(self.m//2)]
            else:# Child Overflow X 
                return node , None
            
            # 자기자신 OverFlow?
            # if len(node.keys) == self.m:
            #     node = splitNode(p_pos,p_node,node)
                
                

        

# if __name__=="__main__":
print("Start")
bt = BTree(3)
kvs = [[1,11],[7,77],[5,55],[6,66],[4,44],[3,33],[2,22],[23,9]]
# for i in random.shuffle([i for i in range(10)]): 

# rand=random.randint(1,50)
# for i in kvs:
#     bt.insert_node(bt.root,i)
# bt.printTree(bt.root)
bt.insert_node(bt.root,[1,11])
bt.insert_node(bt.root,[2,11])
bt.insert_node(bt.root,[3,11])
bt.insert_node(bt.root,[4,11])


Start
[1, 11] []
[2, 11] [[1, 11]]
[3, 11] [[1, 11], [2, 11]]
[4, 11] [[1, 11], [2, 11], [3, 11]]
0


TypeError: '<' not supported between instances of 'int' and 'list'

In [39]:
bt.root

[[1, 11], [2, 11], [3, 11]]